# Manage Metabase configuration for Aroolla


In [ ]:
import requests
import json
API_ENDPOINT = ""
ADMIN_USERNAME = ""
ADMIN_PASSWORD = ""
DB_USER_PASSWORD = ""
DB_TEACHER_PASSWORD = ""
PREFIX = "Aroolla"
CLASSNAME = "3-EEA"
NB_GROUPS = 6
EMAIL_DOMAIN = "aroolla.ch"
groups = ["%s_%s" % (CLASSNAME, chr(65 + i)) for i in range(NB_GROUPS)]
res = requests.post(API_ENDPOINT + "/session",
                    json={"username": ADMIN_USERNAME, "password": ADMIN_PASSWORD})
token = res.json()["id"]
headers = {"X-Metabase-Session": token}
res = requests.get(API_ENDPOINT + "/database", headers=headers)
databases = {db["name"]: db["id"] for db in res.json()["data"]}
res = requests.get(API_ENDPOINT + "/permissions/group", headers=headers)
groups_lookup = {group["name"]: group["id"] for group in res.json()}
res = requests.get(API_ENDPOINT + "/collection", headers=headers)
collection_lookup = {c["name"]: c["id"] for c in res.json()}
res = requests.get(API_ENDPOINT + "/user", headers=headers)
users_lookup = {user["email"]: user["id"] for user in res.json()["data"]}

({"databases": len(databases), "groups": len(groups_lookup), "collections": len(collection_lookup), "users": len(users_lookup)})

## Data Sources


In [ ]:
DB_HOST = "dwh-ro.aroolla.ch"
DB_NAME = "dwh"


def add_data_source(user, password):
    name = "%s %s" % (PREFIX, user)
    if name in databases:
        print("Data source %s already exists" % name)
        return
    payload = {
        "engine": "postgres",
        "name": name,
        "details": {
            "host": DB_HOST,
            "port": "5432",
            "db": DB_NAME,
            "user": user,
            "password": password,
            "schema-filters-patterns": "public",
            "schema-filters-type": "inclusion"
        }
    }
    res = requests.post(API_ENDPOINT + "/database",
                        headers=headers, json=payload)
    db = res.json()
    if "message" in db:
        raise(Exception(db["message"]))
    databases[db["name"]] = db["id"]


add_data_source(CLASSNAME, DB_TEACHER_PASSWORD)
[add_data_source(group, DB_USER_PASSWORD) for group in groups]


In [ ]:
CATEGORY_FIELDS = [
    "Account Code",
    "Account",
    "Category",
    "Classroom",
    "Company",
    "Customer",
    "Destination",
    "Game",
    "Invoice Status",
    "Origin",
    "Product",
    "Supplier",
    "Type",
]


def update_metamodel(db_id):
    res = requests.get(API_ENDPOINT + "/database/%d/fields" %
                       db_id, headers=headers)
    fields = res.json()
    for field in fields:
        if field["name"] in CATEGORY_FIELDS:
            requests.put(API_ENDPOINT + "/field/%d" % field["id"],
                         headers=headers, json={"semantic_type": "type/Category", "has_field_values": "list"})


[update_metamodel(databases[db_name])
 for db_name in databases.keys() if db_name.startswith(PREFIX)]


## Groups, Collections and Permissions


In [ ]:
def create_group(name):
    if name in groups_lookup:
        print("Group %s already exists" % name)
        return
    res = requests.post(API_ENDPOINT + "/permissions/group",
                        headers=headers, json={"name": name})
    group = res.json()
    groups_lookup[group["name"]] = group["id"]


create_group(CLASSNAME)
[create_group(group) for group in groups]

# groupid / dbid .data
res = requests.get(API_ENDPOINT + "/permissions/graph", headers=headers)
graph = res.json()
# remove all permissions for group "All Users"
for key in graph["groups"]["1"]:
    graph["groups"]["1"][key]["data"] = {"schemas": "none", "native": "none"}
for group in groups + [CLASSNAME]:
    graph["groups"][groups_lookup[group]] = {databases["%s %s" % (
        PREFIX, group)]: {"data": {"schemas": "all", "native": "write"}}}

res = requests.put(API_ENDPOINT + "/permissions/graph",
                   headers=headers, json=graph)
graph = res.json()

def create_collection(name, parent_id=None):
    if name in collection_lookup:
        print("Collection %s already exists" % name)
        return
    res = requests.post(API_ENDPOINT + "/collection",
                        headers=headers, json={"name": name, "color": "#509EE3", "parent_id": parent_id})
    collection = res.json()
    collection_lookup[collection["name"]] = collection["id"]


root_project_name = "%s %s" % (PREFIX,  CLASSNAME)
create_collection(root_project_name)
root_project_id = collection_lookup[root_project_name]
create_collection(CLASSNAME, root_project_id)
[create_collection(group, root_project_id) for group in groups]

# groupdid / collectionid|root :  'none' |'read' | 'write'
res = requests.get(API_ENDPOINT + "/collection/graph", headers=headers)
graph = res.json()
# remove all permissions for group "All Users"
for key in graph["groups"]["1"]:
    graph["groups"]["1"][key] = "none"
# each group can write his collection and read the class collection
for group in groups + [CLASSNAME]:
    graph["groups"][groups_lookup[group]] = {
        collection_lookup[group]: "write", root_project_id: "read"}
# teacher can write the class collection
graph["groups"][groups_lookup[CLASSNAME]][root_project_id] = "write"
# teacher can write all groups
for group in groups:
    graph["groups"][groups_lookup[CLASSNAME]
                    ][collection_lookup[group]] = "write"

graph = res = requests.put(
    API_ENDPOINT + "/collection/graph", headers=headers, json=graph)


## Users


In [ ]:
# create companies users with group
def create_user(name, password):
    email = ("%s@%s" % (name, EMAIL_DOMAIN)).lower()
    if email in users_lookup:
        print("User %s already exists" % email)
        # reset password
        requests.put(API_ENDPOINT + "/user/%d/password" % users_lookup[email],
                     headers=headers, json={"password": password})
        return
    user = {
        "email": email,
        "password": password,
    }
    res = requests.post(API_ENDPOINT + "/user", headers=headers, json=user)
    user = res.json()
    if "errors" in user:
        raise(Exception(user["errors"]))
    users_lookup[user["email"]] = user["id"]


create_user(CLASSNAME, DB_TEACHER_PASSWORD)
[create_user(group, DB_USER_PASSWORD) for group in groups]

# update permissions
for group in groups + [CLASSNAME]:
    email = ("%s@%s" % (group, EMAIL_DOMAIN)).lower()
    res = requests.post(API_ENDPOINT + "/permissions/membership",
                        headers=headers, json={"user_id": users_lookup[email], "group_id": groups_lookup[group]})


## Questions Backup and Restore


### Export Questions


In [ ]:
COLLECTION_NAME = "3-EEA"

def replace_field(obj, fields_lookup):
    if isinstance(obj, dict):
        obj = list(obj.values())
    if isinstance(obj, list):
        for item in obj:
            if item == "field":
                if isinstance(obj[1], str):
                    obj[1] = "__NO_REPLACE__%s" % obj[1]
                else:
                    obj[1] = fields_lookup[obj[1]]
            replace_field(item, fields_lookup)


def export_card(card):
    res = requests.get(API_ENDPOINT + "/database/%d?include=tables.fields" %
                       card["dataset_query"]["database"], headers=headers)
    database = res.json()
    card["dataset_query"]["database"] = database["name"]

    if card["dataset_query"]["type"] == "query":
        # only support limited queries which we use.
        # TODO check join queries?
        if "source-table" in card["dataset_query"]["query"]:
            source_table = card["dataset_query"]["query"]["source-table"]
        else:
            source_table = card["dataset_query"]["query"]["source-query"]["source-table"]

        table = [table for table in database["tables"]
                 if table["id"] == source_table][0]
        if "source-table" in card["dataset_query"]["query"]:
            card["dataset_query"]["query"]["source-table"] = table["name"]
        else:
            card["dataset_query"]["query"]["source-query"]["source-table"] = table["name"]
        fields_lookup = {field["id"]: field["name"]
                         for field in table["fields"]}
        replace_field(card, fields_lookup)
    if card["dataset_query"]["type"] == "native":
        fields_lookup = {}
        for table in database["tables"]:
            for field in table["fields"]:
                fields_lookup[field["id"]] = "%s.%s" % (table["name"], field["name"])
        replace_field(card["dataset_query"]["native"]["template-tags"], fields_lookup)

    backup_card = {
        "name": card["name"],
        "description": card["description"],
        "dataset_query": card["dataset_query"],
        "display": card["display"],
        "visualization_settings": card["visualization_settings"],
        "parameters": card["parameters"],
        "result_metadata": card["result_metadata"],
    }
    return backup_card


res = requests.get(API_ENDPOINT + "/card", headers=headers)
cards = [export_card(card) for card in res.json(
) if card["collection_id"] == collection_lookup[COLLECTION_NAME]]

with open("cards.json", "w", encoding='utf-8') as f:
    json.dump(cards, f, indent=2, ensure_ascii=False)


### Import Questions


In [ ]:
DATABASE_NAME = "%s %s" % (PREFIX, CLASSNAME)
COLLECTION_NAME = "test"
GAME_NAME = "Extended1-3-EEA"

def replace_field(obj, fields_lookup, parent_obj=None):
    if isinstance(obj, dict):
        obj = list(obj.values())
    if isinstance(obj, list):
        for item in obj:
            if item == "field":
                if obj[1] == "game" and len(parent_obj) >= 3:
                    parent_obj[2] = GAME_NAME
                if obj[1] == "classroom" and len(parent_obj) >= 3:
                    parent_obj[2] = CLASSNAME
                if obj[1].startswith("__NO_REPLACE__"):
                    obj[1] = obj[1].replace("__NO_REPLACE__", "")
                else:
                    obj[1] = fields_lookup[obj[1]]
                

            replace_field(item, fields_lookup, obj)


def import_card(card):
    res = requests.get(API_ENDPOINT + "/database/%d?include=tables.fields" %
                       databases[DATABASE_NAME], headers=headers)
    database = res.json()
    card["dataset_query"]["database"] = database["id"]
    card["collection_id"] = collection_lookup[COLLECTION_NAME]

    if card["dataset_query"]["type"] == "query":
        if "source-table" in card["dataset_query"]["query"]:
            source_table = card["dataset_query"]["query"]["source-table"]
        else:
            source_table = card["dataset_query"]["query"]["source-query"]["source-table"]

        table = [table for table in database["tables"]
                 if table["name"] == source_table][0]
        if "source-table" in card["dataset_query"]["query"]:
            card["dataset_query"]["query"]["source-table"] = table["id"]
        else:
            card["dataset_query"]["query"]["source-query"]["source-table"] = table["id"]
        fields_lookup = {field["name"]: field["id"]
                         for field in table["fields"]}
        replace_field(card, fields_lookup)

    if card["dataset_query"]["type"] == "native":
        # revert fields for variables
        fields_lookup = {}
        for table in database["tables"]:
            for field in table["fields"]:
                fields_lookup["%s.%s" % (table["name"], field["name"])] = field["id"]
        replace_field(card["dataset_query"]["native"]["template-tags"], fields_lookup)
        
        # udpate game and classroom filters in native query
        if "game" in card["dataset_query"]["native"]["template-tags"]:
            card["dataset_query"]["native"]["template-tags"]["game"]["default"][0] = GAME_NAME
        if "classroom" in card["dataset_query"]["native"]["template-tags"]:
            card["dataset_query"]["native"]["template-tags"]["classroom"]["default"][0] = CLASSNAME
        for param in card["parameters"]:
            if param["slug"] == "game":
                param["default"][0] = GAME_NAME
            if param["slug"] == "classroom":
                param["default"][0] = CLASSNAME

    if card["name"] in cards_lookup:
        requests.put(API_ENDPOINT + "/card/%d" % cards_lookup[card["name"]], headers=headers, json=card)
    else:
        requests.post(API_ENDPOINT + "/card", headers=headers, json=card)

res = requests.get(API_ENDPOINT + "/card", headers=headers)
cards_lookup = {card["name"]: card["id"] for card in res.json(
) if card["collection_id"] == collection_lookup[COLLECTION_NAME]}

# load files
with open("cards.json", "r", encoding='utf-8') as f:
    cards = json.load(f)
    [import_card(card) for card in cards]


### Export Dashboards

In [ ]:
COLLECTION_NAME = "test"

res = requests.get(API_ENDPOINT + "/dashboard", headers=headers)
dashboards = [dashboard for dashboard in res.json() if dashboard["collection_id"] == collection_lookup[COLLECTION_NAME]]
res = requests.get(API_ENDPOINT + "/dashboard/1", headers=headers)
res.json()
# requires mapping query, table, fields and remapp question ids
# or does it recreate question/card?